In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import json
from sqlalchemy import create_engine, inspect, Table, Column
import pandas as pd
from args_reader import read_args
from scicrunch_client_fast_mod import scicrunch
import sys
import time
from interlex_sql import interlex_sql
import re
from collections import defaultdict

args = read_args(api_key='../production_api_scicrunch_key.txt', engine_key='../production_engine_scicrunch_key.txt', production=True)
sql = interlex_sql(engine_key=args.engine_key)
sci = scicrunch(api_key=args.api_key, base_path=args.base_path, engine_key=args.engine_key)

In [2]:
tier1 = '/home/troy/Desktop/NDA/nda-cdes-first-tier-only-update-ready.json'
tier2 = '/home/troy/Desktop/NDA/complete_nda_tier2_production_ref.json'
nda_names = list(json.load(open(tier1)).keys()) + list(json.load(open(tier2)).keys())
tier1_data = json.load(open(tier1))
tier2_data = json.load(open(tier2))

In [50]:
tier2_data

{'subjectkey (Abnormal Involuntary Movement Scale)': {'label': 'subjectkey (Abnormal Involuntary Movement Scale)',
  'definition': 'The NDAR Global Unique Identifier (GUID) for research subject',
  'type': 'cde',
  'annotation_tid': 15074,
  'value': ['id : 2039689'],
  'superclass_label': 'Abnormal Involuntary Movement Scale',
  'superclass_ilx': 'ilx_0339882',
  'superclass_tid': 258573},
 'src_subject_id (Abnormal Involuntary Movement Scale)': {'label': 'src_subject_id (Abnormal Involuntary Movement Scale)',
  'definition': "Subject ID how it's defined in lab/project",
  'type': 'cde',
  'annotation_tid': 15074,
  'value': ['id : 2039690'],
  'superclass_label': 'Abnormal Involuntary Movement Scale',
  'superclass_ilx': 'ilx_0339882',
  'superclass_tid': 258573},
 'interview_date (Abnormal Involuntary Movement Scale)': {'label': 'interview_date (Abnormal Involuntary Movement Scale)',
  'definition': 'Date on which the interview/genetic test/sampling/imaging/biospecimen was completed

In [3]:
labels_to_id = sql.get_labels_to_ids_dict()

In [4]:
nda_ids = []
for label in nda_names:
    label = label.lower().strip().replace('"', '').replace("'", '')
    if not labels_to_id.get(label):
        sys.exit(label)
    else:
        nda_ids.append(labels_to_id[label]['cde'])

In [33]:
terms_df = sql.get_terms()
annotations_df = sql.get_annotations()

In [6]:
id_to_label = sql.get_ids_to_labels_dict()

In [7]:
annotations_df.loc[annotations_df.item_id == 230000].to_dict('list')

{'item_id': [230000, 230000, 230000, 230000, 230000, 230000, 230000, 230000],
 'label': ['vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)',
  'vine_com_busletter (Vineland I)'],
 'ilx': ['ilx_0312138',
  'ilx_0312138',
  'ilx_0312138',
  'ilx_0312138',
  'ilx_0312138',
  'ilx_0312138',
  'ilx_0312138',
  'ilx_0312138'],
 'anno_tid': [15074, 15074, 15074, 15074, 15074, 15074, 15074, 15074],
 'anno_label': ['hasConstraint',
  'hasConstraint',
  'hasConstraint',
  'hasConstraint',
  'hasConstraint',
  'hasConstraint',
  'hasConstraint',
  'hasConstraint'],
 'anno_ilx': ['ilx_0115071',
  'ilx_0115071',
  'ilx_0115071',
  'ilx_0115071',
  'ilx_0115071',
  'ilx_0115071',
  'ilx_0115071',
  'ilx_0115071'],
 'tid': [230000, 230000, 230000, 230000, 230000, 230000, 230000, 230000],
 'annotatio

In [23]:
nda_ids[0]['cde']

15076

In [59]:
annotations_df.loc[annotations_df.label == 'eventname (ABCD Children&#39;s Report of Parental Behavioral Inventory)']

,item_id,label,ilx,anno_tid,anno_label,anno_ilx,tid,annotation_tid,value
12891,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,required : Recommended
12892,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,condition : None
12893,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,filterElement : None
12894,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,type : String
12895,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,size : 60
12896,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,valueRange : None
12897,15813,eventname (ABCD Children&#39;s Report of Paren...,ilx_0115810,15074,hasConstraint,ilx_0115071,15813,15074,notes : None


In [34]:
%%time
cdes_no_annotations = []
for id in nda_ids[:]:
    hit = annotations_df.loc[annotations_df.tid == id]
    if hit.empty:
        cdes_no_annotations.append({'id':id, 'label':id_to_label[id]})
print(len(cdes_no_annotations))
#with open('/home/troy/Desktop/NDA/missing_annotaions.json', 'w') as f:
#json.dump(cdes_no_annotations, f, indent=4)

0
CPU times: user 5min 29s, sys: 20.3 ms, total: 5min 29s
Wall time: 5min 29s


In [67]:
for id in nda_ids:
    if id == 15813:
        print('yas')

In [72]:
#%%time
cdes_incomplete_annotations = []
for id in nda_ids[:]:
    if id != 15813: continue
    print(id)
    hit = annotations_df.loc[annotations_df.tid == id]
    if hit.empty:
        pass#cdes_no_annotations.append({'id':id, 'label':id_to_label[id]})
    else:
        vs = hit.to_dict('list')['value']
        if '(abcd children' in id_to_label[id].lower():
            print(id_to_label[id])
            sys.exit(vs)
        #if 'id' not in ''.join(vs) and '(' in id_to_label[id] and id > 16000:
            #print(vs)
        #if len(hit.to_dict('list')['value']) not in [5, 8]:
        #    cdes_incomplete_annotations.append({'id':id, 'label':id_to_label[id]})
        #    sys.exit(cdes_incomplete_annotations)
hit           
#print(len(cdes_incomplete_annotations))
#with open('/home/troy/Desktop/NDA/incomplete_annotaions.json', 'w') as f:
#    json.dump(cdes_incomplete_annotations, f, indent=4)

[]

In [20]:
missing_annotatons_data = json.load(open('/home/troy/Desktop/NDA/missing_annotaions.json'))
incomplete_annotations_data = json.load(open('/home/troy/Desktop/NDA/incomplete_annotaions.json'))

In [25]:
tier2_data

{'subjectkey (Abnormal Involuntary Movement Scale)': {'label': 'subjectkey (Abnormal Involuntary Movement Scale)',
  'definition': 'The NDAR Global Unique Identifier (GUID) for research subject',
  'type': 'cde',
  'annotation_tid': 15074,
  'value': ['condition : None',
   'filterElement : None',
   'id : 2039689',
   'notes : None',
   'required : Required',
   'size : None',
   'type : GUID',
   'valueRange : NDAR*'],
  'superclass_label': 'Abnormal Involuntary Movement Scale',
  'superclass_ilx': 'ilx_0339882',
  'superclass_tid': 258573},
 'src_subject_id (Abnormal Involuntary Movement Scale)': {'label': 'src_subject_id (Abnormal Involuntary Movement Scale)',
  'definition': "Subject ID how it's defined in lab/project",
  'type': 'cde',
  'annotation_tid': 15074,
  'value': ['condition : None',
   'filterElement : None',
   'id : 2039690',
   'notes : None',
   'required : Required',
   'size : 20',
   'type : String',
   'valueRange : None'],
  'superclass_label': 'Abnormal Invol

In [30]:
''' Fix missing annotations '''
annotations = []
for e in missing_annotatons_data:
    if tier1_data.get(e['label']):
        t1 = tier1_data[e['label']]
        for v in t1['value']:
            annotations.append({'tid':e['id'], 'annotation_tid':15074, 'value':v})
    elif tier2_data.get(e['label']):
        t2 = tier2_data[e['label']]
        for v in t2['value']:
            annotations.append({'tid':e['id'], 'annotation_tid':15074, 'value':v})
    else:
        sys.exit(e)
len(annotations)
with open('/home/troy/Desktop/NDA/missing_annotaions_client_ready.json', 'w') as f:
    json.dump(annotations, f, indent=4)

In [53]:
annotations = []
for e in incomplete_annotations_data:
    
    if tier1_data.get(e['label']):
        t1 = tier1_data[e['label']]
        for v in t1['value']:
            annotations.append({'tid':str(e['id']), 'annotation_tid':15074, 'value':str(v)})
    
    elif tier2_data.get(e['label']):
        t2 = tier2_data[e['label']]
        for v in t2['value']:
            annotations.append({'tid':str(e['id']), 'annotation_tid':15074, 'value':str(v)}) 
        print(e['label'])
    else:
        sys.exit(e)
        
len(annotations)
with open('/home/troy/Desktop/NDA/incomplete_annotaions_client_ready.json', 'w') as f:
    json.dump(annotations, f, indent=4)

bdi1 (Beck Depression Inventory)
bdi4 (Beck Depression Inventory)
bdi5 (Beck Depression Inventory)
bdi6 (Beck Depression Inventory)
bdi8 (Beck Depression Inventory)
bdi9 (Beck Depression Inventory)
bdi10 (Beck Depression Inventory)
bdi11 (Beck Depression Inventory)
bdi12 (Beck Depression Inventory)
bdi14 (Beck Depression Inventory)
bdi15 (Beck Depression Inventory)
bdi16 (Beck Depression Inventory)
bdi19 (Beck Depression Inventory)
bdi_cat (Beck Depression Inventory)
bd_004 (Beck Depression Inventory)
carp15 (25-year Carpenter Prognostic Update)
carp16 (25-year Carpenter Prognostic Update)
carp17 (25-year Carpenter Prognostic Update)
carp18 (25-year Carpenter Prognostic Update)
grade_highed (AASPIRE Health Integration)
aut_posteval_effect (AASPIRE Health Integration)
sup_prepers_comm (AASPIRE Health Integration)
cash_choice_task (ABCD Cash Choice Task)
devhx_2_p_dk (ABCD Developmental History Questionnaire)
devhx_2b_p_dk (ABCD Developmental History Questionnaire)
devhx_3_p (ABCD Develo

In [2]:
## So the shortNames list is the only complete data for some unknown reason at the moment; best to just use that to make pure annotations and update defintions
tier2 = json.load(open('/home/troy/Desktop/NDA/nda_shortName_data.json', 'r'))

In [16]:
complete_nda_tier2_production_ref = defaultdict(dict)
for k, v in tier2.items():
    print(k)
    for d in v['dataElements']:
        print(d)
        annotations = []
        for annotation_key, annotations_values in v.items():
            if isinstance(annotations_values, list) and annotations_values:
                #print(annotation_key, annotations_values)
                value = annotation_key + ' : ' + '; '.join(annotations_values)
            else:
                value = annotation_key + ' : ' + annotations_values
            annotations.append(value)
        complete_nda_tier2_production_ref[k].update({'value':annotations})
        break
    break
complete_nda_tier2_production_ref

12-Item Grit Scale
{'aliases': [], 'condition': None, 'dataElementId': 1, 'description': 'The NDAR Global Unique Identifier (GUID) for research subject', 'filterElement': None, 'id': 2081167, 'name': 'subjectkey', 'notes': None, 'position': 1, 'required': 'Required', 'size': None, 'translations': [], 'type': 'GUID', 'valueRange': 'NDAR*'}


TypeError: sequence item 0: expected str instance, dict found